In [1]:
import PyPDF2 as p
import config as conf
import os
import pandas as pd
import tabula.io as tb
from extracttools import good_match

In [2]:
def polish(df: pd.DataFrame) -> pd.DataFrame:
    # remove \r characters
    df = df.rename(lambda s:s.replace('\r',' '),axis='columns') # remove '\r' from headers
    df = df.replace('\r','',regex=True) # replace all '\r' throughout
    # approximate headers
    cols = df.columns
    newcols = [good_match(col,conf.CANON_HEADERS) for col in cols]
    df.columns = newcols
    print(df.columns)
    return df
def is_header(row):
    if sum([bool(any([fuzz.token_sort_ratio(cell,target) > 60 for target in ref])) for cell in head.loc[row]]) < 2:
        """ nonsense case -> treat as overflow """
        return True
    else:
        return False
def separate_overflow(df,ind,page_range) -> list[pd.DataFrame]:
    # step 1 - find where table actually begins
    start = 0
    for row in range(len(df)):
        print(row)
        if is_header(df.loc[row]):
            start = row
            print('is header')
    if start == 0:
        if not is_header(df.loc[0]):
            if not df.empty: # case 2: no row resembling a header found
                df.columns = conf.CANON_HEADERS # could result in mismatched headers
                raise(BaseException('Could result in mismatched headers on p.'+str(int(page_range.split('-')[0])+ind)))
    overflow = df.loc[0:start]
    df.drop([i for i in range(0,start)])
    df.columns = df.loc[start]
    df.reset_index()
    return [df,overflow]
def fix_overflow(dfs: list[pd.DataFrame],page_range: str) -> list[pd.DataFrame]:
    print('SHOW UP DAMMIT!')
    for i,df in enumerate(dfs[1:]):
        print('why isn\'t this printing?')
        [df,overflow] = separate_overflow(df,i,page_range)
        # if sum([col != '' for col in overflow]) <= 2: # merging with last row on prev page TODO
        if 0 == sum([s in overflow['observations and recommendations'][0] for s in conf.BULLET_STRS],[s in overflow['audit observation'][0] for s in conf.BULLET_STRS]):
            dfs[i-1].loc[-1] += overflow
        dfs[i-1].append(overflow)
    print('OVERFLOW: '+overflow)
    return dfs
def longest_seq(li: list):
    longest = []
    seq = [li[0]]
    longest_streak = 0
    streak = 0
    for i in range(len(li)-1):
        if li[i] == li[i+1]-1:
            seq = seq + [li[i+1]]
            streak = streak + 1
        else:
            streak = 0
            seq = [li[i+1]]
        if longest_streak < streak:
            longest_streak = streak
            longest = seq
    return longest
def get_pg_rng(pdf_url):
    pdf = p.PdfFileReader(pdf_url)
    n = pdf.numPages
    outp = ''
    pgs = list()
    for pg in range(n):
        page = pdf.getPage(pg)
        content = page.extractText()
        if all([target in content.lower() for target in conf.TARGET_SENTENCE]):
            pgs = pgs + [pg]
    if len(pgs) < 1:
        print('Warning: No pages contain tables: '+pdf_url)
        return '0'
    elif len(pgs) < 2: 
        pgs = [pg for pg in range(pgs[0],n)]
        print('Warning: Only 1 page was detected containing target title')
        return str(min(pgs)+1) + '-' + str(max(pgs)+1)
    elif longest_seq(pgs) == []:
        print('Warning: May have missed a document')
        return '0' # when this occurred, it was the wrong table
    else:
        pgs = longest_seq(pgs) # assume that the area with the most instances of the target sequence words
                           # is where Part IV actually begins as opposed to some front matter or table of contents
        pgs = [pg for pg in range(pgs[0],n)] # scrape starting from official beginning of Part IV until the end of the document
        return str(min(pgs)+1) + '-' + str(max(pgs)+1)
def city(df:pd.DataFrame,pdf_url:str)->pd.DataFrame:
    df = df.assign(city=''.join([c for c in ((pdf_url.split('/')[-1])[2:].split('_'))[0].replace('-','') if ord(c) >= 65]))
    for cell in df['city']:
        if cell in list(conf.ABBREV_CITY_NAMES.keys()):
            df = df.assign(city = conf.ABBREV_CITY_NAMES[cell])
    return df
def year(df:pd.DataFrame,pdf_url:str)->pd.DataFrame:
    return df.assign(year=''.join([c for c in ((pdf_url.split('/')[-1])[2:].split('_'))[0].replace('-','') if ord(c) < 65]))

In [3]:
def extract(pdf_url) -> pd.DataFrame:
    pg_rng = get_pg_rng(pdf_url)
    if pg_rng == '0':
        return
    print(pg_rng)
    dfs = tb.read_pdf(input_path = pdf_url, output_format = 'dataframe', pages = pg_rng, lattice = True, multiple_tables = True)
    dfs = [polish(df) for df in dfs]
    print('fix_overflow print()\'s won\'t print')
    dfs = [dfs[0]]+[fix_overflow(dfs,pg_rng) for df in dfs[1:]]
    print(dfs[0]['audit observation'])
    dfs = [year(city(df,pdf_url),pdf_url) for df in dfs]
    out = pd.concat(dfs,ignore_index=True,axis=0)
    return out

In [4]:
df = extract('../test_data/pdf/01-IGACOS2010_Audit_Report.pdf')

56-75
Index(['audit observation', 'recommendations', 'ref', 'management action',
       'status of implementation', 'reasons for partial/non-implementation'],
      dtype='object')
fix_overflow print()'s won't print
0     Non-settingupofRPT/SET Receivables1.RealProper...
1     UnrecordedInterest Income2.   Failure to recor...
2                                                   NaN
3                           AuditObservations (CY 2009)
4     Understated InsuranceExpense.    The expired p...
5     Uncancelled StaleChecks4.Stalechecksamounting ...
6                           AuditObservations (CY 2009)
7     Unadjusted InventoryAccount With No StockBalan...
8     Unreconciled Differencein Inventory Reports6.T...
9                           AuditObservations (CY 2009)
10    ErroneousClassification ofPayable Accounts7.  ...
11    Dormant OtherReceivables Account8.Despite  pre...
12                          AuditObservations (CY 2009)
13    overstatement of agencyassets due to theinclus...
